# FEP Quickstart Guide

This notebook provides a simple introduction to the Flexible Evaluation Protocol (FEP) - a vendor-neutral, schema-driven standard for evaluating any system that produces complex or variable outputs.

## What is FEP?

FEP provides:
- **Portable data formats** for test cases, system outputs, and results
- **Pluggable "checks"** for boolean, numeric, categorical, and free-text assessments
- **JSONPath expressions** for flexible data extraction and comparison

Let's see it in action!

## Installation and Setup

If you haven't already, install the flex-evals package into your environment: `pip install flex-evals`

In [1]:
# Import the core FEP components
from flex_evals import (
    evaluate,
    TestCase,
    Output,
    ExactMatchCheck,
    ContainsCheck,
    RegexCheck,
    ThresholdCheck,
)
import json

## Example 1: Simple Text Evaluation

Let's start with a basic question-answering evaluation where we check if a system gives the correct answer.

In [2]:
# Define our test cases
test_cases = [
    TestCase(
        id="geography_001",
        input="What is the capital of France?",
        expected="Paris",
    ),
    TestCase(
        id="math_001",
        input="What is 2 + 2?",
        expected="4",
    ),
]

# Simulate system outputs (in reality, these would come from your LLM/API)
outputs = [
    Output(value="Paris"),
    Output(value="Four"),
]

print("Test Cases:")
for tc in test_cases:
    print(f"  {tc.id}: {tc.input} → Expected: {tc.expected}")

print("\nSystem Outputs:")
for i, output in enumerate(outputs):
    print(f"  {test_cases[i].id}: {output.value}")

Test Cases:
  geography_001: What is the capital of France? → Expected: Paris
  math_001: What is 2 + 2? → Expected: 4

System Outputs:
  geography_001: Paris
  math_001: Four


In [3]:
# Define checks to evaluate the outputs
#
# FEP supports two different patterns for organizing checks:
#
# 1. **Shared Checks (List[Check])** - Same checks applied to ALL test cases
#    This is what we're using here - the exact_match check will be applied to both test cases
#    This is the "1-to-many" pattern from the FEP specification
#
# 2. **Per-Test-Case Checks (List[List[Check]])** - Each test case has its own checks
#    Example: [[check1, check2], [check3, check4]] where first list applies to test_cases[0]
#    This allows different evaluation criteria per test case
#
# Note: This Python implementation also supports a convenience feature where you can
# pass checks=None and embed checks directly in TestCase objects using TestCase.checks,
# but this is not part of the core FEP specification - it's a Python-specific extension.
#
# The shared pattern (used below) is perfect when you want to apply the same
# evaluation criteria across all test cases, like checking that all responses
# match their expected values.

checks = [
    # this check can be shared across all test cases since it referencesthe test case's expected
    # value dynamically using JSONPath
    ExactMatchCheck(
        actual="$.output.value",        # JSONPath to extract actual output
        expected="$.test_case.expected", # JSONPath to extract expected output
        case_sensitive=False,            # Allow case-insensitive matching
    ),
]

print("Evaluation Check:")
print("  Type: exact_match")
print(f"  Case Sensitive: {checks[0].case_sensitive}")
print(f"  Pattern: Shared checks (same check applied to all {len(test_cases)} test cases)")

Evaluation Check:
  Type: exact_match
  Case Sensitive: False
  Pattern: Shared checks (same check applied to all 2 test cases)


In [4]:
# Run the evaluation
result = evaluate(
    test_cases=test_cases,
    outputs=outputs,
    checks=checks,
)

print(f"Evaluation completed in {(result.completed_at - result.started_at).total_seconds():.3f} seconds")  # noqa: E501
print(f"Status: {result.status}")
print(f"Summary: {result.summary.completed_test_cases}/{result.summary.total_test_cases} test cases completed")  # noqa: E501


Evaluation completed in 0.011 seconds
Status: completed
Summary: 2/2 test cases completed


In [5]:
# Examine the detailed results
print("Detailed Results:")
print("=" * 50)

for test_result in result.results:
    print(f"\nTest Case: {test_result.execution_context.test_case.id}")
    print(f"Status: {test_result.status}")
    for check_result in test_result.check_results:
        print(f"\n  Check: {check_result.check_type}")
        print(f"  Passed: {check_result.results['passed']}")
        print(f"  Actual: '{check_result.resolved_arguments['actual']['value']}'")
        print(f"  Expected: '{check_result.resolved_arguments['expected']['value']}'")

Detailed Results:

Test Case: geography_001
Status: completed

  Check: exact_match
  Passed: True
  Actual: 'Paris'
  Expected: 'Paris'

Test Case: math_001
Status: completed

  Check: exact_match
  Passed: False
  Actual: 'Four'
  Expected: '4'


## Example 2: Multiple Check Types

Let's use different types of checks to evaluate various aspects of system outputs.

In [6]:
# Test case for a coding question
coding_test = TestCase(
    id="coding_001",
    input="Write a Python function that returns the square of a number",
    metadata={"difficulty": "easy", "language": "python"},
)

# Simulated system output with structured data
coding_output = Output(
    value={
        "code": "def square(x):\n    return x * x",
        "explanation": "This function takes a number x and returns its square by multiplying it by itself.",  # noqa: E501
        "confidence": 0.95,
    },
    metadata={"execution_time_ms": 150, "model": "gpt-4"},
)

print("Test Case:")
print(f"  ID: {coding_test.id}")
print(f"  Input: {coding_test.input}")
print(f"  Metadata: {coding_test.metadata}")

print("\nSystem Output:")
print(f"  Code: {coding_output.value['code'][:30]}...")
print(f"  Confidence: {coding_output.value['confidence']}")
print(f"  Execution Time: {coding_output.metadata['execution_time_ms']}ms")

Test Case:
  ID: coding_001
  Input: Write a Python function that returns the square of a number
  Metadata: {'difficulty': 'easy', 'language': 'python'}

System Output:
  Code: def square(x):
    return x * ...
  Confidence: 0.95
  Execution Time: 150ms


In [7]:
# Define multiple checks for different aspects
multi_checks = [
    # Check 1: Code contains the word "def" (function definition)
    ContainsCheck(
        text="$.output.value.code",
        phrases=["def"],
    ),

    # Check 2: Code matches function pattern with regex
    RegexCheck(
        text="$.output.value.code",
        pattern=r"def\s+\w+\s*\(",  # Function definition pattern
        flags={"case_insensitive": True},
    ),

    # Check 3: Confidence is above threshold
    ThresholdCheck(
        value="$.output.value.confidence",
        min_value=0.8,
    ),

    # Check 4: Execution time is reasonable
    ThresholdCheck(
        value="$.output.metadata.execution_time_ms",
        max_value=1000,
    ),
]

print("Evaluation Checks:")
for i, check in enumerate(multi_checks, 1):
    check_type = check.__class__.__name__.replace('Check', '').lower()
    print(f"  {i}. {check_type} - {list(check.model_dump().keys())}")

Evaluation Checks:
  1. contains - ['version', 'text', 'phrases', 'case_sensitive', 'negate']
  2. regex - ['version', 'text', 'pattern', 'negate', 'flags']
  3. threshold - ['version', 'value', 'min_value', 'max_value', 'min_inclusive', 'max_inclusive', 'negate']
  4. threshold - ['version', 'value', 'min_value', 'max_value', 'min_inclusive', 'max_inclusive', 'negate']


In [8]:
# Run the multi-check evaluation
multi_result = evaluate(
    test_cases=[coding_test],
    outputs=[coding_output],
    checks=multi_checks,
)

print("Multi-Check Evaluation Results:")
print("=" * 40)

test_result = multi_result.results[0]
print(f"\nTest Case: {test_result.execution_context.test_case.id}")
print(f"Overall Status: {test_result.status}")
print(f"Checks Passed: {test_result.summary.completed_checks}/{test_result.summary.total_checks}")

print("\nIndividual Check Results:")
for i, check_result in enumerate(test_result.check_results, 1):
    status_icon = "✓" if check_result.results.get('passed', False) else "✗"
    print(f"  {i}. {status_icon} {check_result.check_type}: {check_result.results.get('passed', 'N/A')}")  # noqa: E501

    # Show resolved values for context
    if 'value' in check_result.resolved_arguments:
        value = check_result.resolved_arguments['value']['value']
        print(f"     Value: {value}")

Multi-Check Evaluation Results:

Test Case: coding_001
Overall Status: completed
Checks Passed: 4/4

Individual Check Results:
  1. ✓ contains: True
  2. ✓ regex: True
  3. ✓ threshold: True
     Value: 0.95
  4. ✓ threshold: True
     Value: 150


## Example 3: Complex Structured Output

FEP excels at evaluating complex, nested outputs using JSONPath expressions.

In [9]:
# Test case for an AI agent that needs to plan a task
agent_test = TestCase(
    id="agent_001",
    input="Plan a trip to Paris for 3 days",
    expected={
        "duration_days": 3,
        "destination": "Paris",
    },
)

# Complex structured output from an AI agent
agent_output = Output(
    value={
        "plan": {
            "destination": "Paris, France",
            "duration_days": 3,
            "budget_usd": 1500,
            "itinerary": [
                {"day": 1, "activities": ["Visit Eiffel Tower", "Louvre Museum"]},
                {"day": 2, "activities": ["Notre-Dame", "Seine River Cruise"]},
                {"day": 3, "activities": ["Versailles", "Shopping"]},
            ],
        },
        "tools_used": ["travel_search", "hotel_booking", "activity_planner"],
        "confidence": 0.92,
        "reasoning": "Created a balanced 3-day itinerary covering major Paris attractions",
    },
)

print("Complex Agent Output:")
print(json.dumps(agent_output.value, indent=2)[:300] + "...")

Complex Agent Output:
{
  "plan": {
    "destination": "Paris, France",
    "duration_days": 3,
    "budget_usd": 1500,
    "itinerary": [
      {
        "day": 1,
        "activities": [
          "Visit Eiffel Tower",
          "Louvre Museum"
        ]
      },
      {
        "day": 2,
        "activities": [
      ...


In [10]:
# Define checks using JSONPath for complex data extraction
complex_checks = [
    # Check destination contains "Paris"
    ContainsCheck(
        text="$.output.value.plan.destination",
        phrases=["Paris"],
        case_sensitive=False,
    ),

    # Check duration is exactly 3 days
    ExactMatchCheck(
        actual="$.output.value.plan.duration_days",
        expected="$.test_case.expected.duration_days",
    ),

    # Check that budget is reasonable (between $500-$3000)
    ThresholdCheck(
        value="$.output.value.plan.budget_usd",
        min_value=500,
        max_value=3000,
    ),

    # Check that itinerary has 3 days
    ThresholdCheck(
        value="$.output.value.plan.itinerary.length",  # JSONPath length function
        min_value=3,
        max_value=3,
    ),

    # Check that agent used appropriate tools
    ContainsCheck(
        text="$.output.value.tools_used",
        phrases=["travel_search"],
    ),
]

print("Complex Evaluation Checks:")
for i, check in enumerate(complex_checks, 1):
    check_type = check.__class__.__name__.replace('Check', '').lower()
    key_arg = next(iter(check.model_dump().keys()))
    print(f"  {i}. {check_type}: {key_arg} = {getattr(check, key_arg)}")

Complex Evaluation Checks:
  1. contains: version = None
  2. exactmatch: version = None
  3. threshold: version = None
  4. threshold: version = None
  5. contains: version = None


In [11]:
# Run the complex evaluation
complex_result = evaluate(
    test_cases=[agent_test],
    outputs=[agent_output],
    checks=complex_checks,
)

print("Complex Evaluation Results:")
print("=" * 40)

test_result = complex_result.results[0]
print(f"Test Case: {test_result.execution_context.test_case.id}")
print(f"Overall Status: {test_result.status}")
print(f"Success Rate: {test_result.summary.completed_checks}/{test_result.summary.total_checks}")

print("\nDetailed Check Results:")
for i, check_result in enumerate(test_result.check_results, 1):
    passed = check_result.results.get('passed', False)
    status_icon = "✓" if passed else "✗"
    print(f"\n  {i}. {status_icon} {check_result.check_type}")

    # Show what was actually evaluated
    for arg_name, arg_data in check_result.resolved_arguments.items():
        if 'jsonpath' in arg_data:
            print(f"     {arg_name}: {arg_data['value']} (from {arg_data['jsonpath']})")
        else:
            print(f"     {arg_name}: {arg_data['value']}")

Complex Evaluation Results:
Test Case: agent_001
Overall Status: error
Success Rate: 4/5

Detailed Check Results:

  1. ✓ contains
     text: Paris, France (from $.output.value.plan.destination)
     phrases: ['Paris']
     case_sensitive: False
     negate: False

  2. ✓ exact_match
     actual: 3 (from $.output.value.plan.duration_days)
     expected: 3 (from $.test_case.expected.duration_days)
     case_sensitive: True
     negate: False

  3. ✓ threshold
     value: 1500 (from $.output.value.plan.budget_usd)
     min_inclusive: True
     max_inclusive: True
     negate: False
     min_value: 500.0
     max_value: 3000.0

  4. ✗ threshold

  5. ✓ contains
     text: ['travel_search', 'hotel_booking', 'activity_planner'] (from $.output.value.tools_used)
     phrases: ['travel_search']
     case_sensitive: True
     negate: False


## Key Takeaways

From these examples, you can see how FEP provides:

1. **Flexible Data Extraction**: JSONPath expressions like `$.output.value.plan.destination` let you extract data from anywhere in your outputs

2. **Multiple Check Types**: 
   - `exact_match`: For precise comparisons
   - `contains`: For substring/phrase checking
   - `regex`: For pattern matching
   - `threshold`: For numeric bounds checking

3. **Comprehensive Results**: Every evaluation provides detailed results with:
   - What was compared (`resolved_arguments`)
   - Pass/fail status for each check
   - Aggregate statistics
   - Error handling

4. **Vendor Neutrality**: FEP works with any system that produces outputs - LLMs, APIs, traditional software, etc.

## Next Steps

- Check out the advanced examples for YAML-based test case management
- Explore semantic similarity and LLM judge checks for more sophisticated evaluations
- Learn about batch evaluation and result analysis techniques
